In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import pymedphys.electronfactors as electronfactors

In [ ]:
data_filename = r'S:\Physics\RCCC Specific Files\Dosimetry\Elekta_EFacs\electron_factor_measured_data.csv'
data = pd.read_csv(data_filename)

In [ ]:
width_data = data['Width (cm @ 100SSD)']
length_data = data['Length (cm @ 100SSD)']
factor_data = data['RCCC Inverse factor (dose open / dose cutout)']

p_on_a_data = electronfactors.convert2_ratio_perim_area(
    width_data, length_data)

In [ ]:
rect_width = 5 / .95
rect_length = 8 / .95

In [ ]:
x = [
    -rect_width/2, rect_width/2, rect_width/2, -rect_width/2, -rect_width/2
]

y = [
    rect_length/2, rect_length/2, -rect_length/2, -rect_length/2, rect_length/2
]

In [ ]:
rect_width

In [ ]:
width, length = electronfactors.model_and_display(
    x, y
)

In [ ]:
assert np.allclose(rect_width, width)

In [ ]:
p_on_a = electronfactors.convert2_ratio_perim_area(width, length)

In [ ]:
energy = 12
applicator = 10
ssd = 100

reference = (
    (data['Energy (MeV)'] == energy) &
    (data['Applicator (cm)'] == applicator) &
    (data['SSD (cm)'] == ssd)
)

electronfactors.spline_model_with_deformability(
    width,
    p_on_a,
    width_data[reference], 
    p_on_a_data[reference],
    factor_data[reference]
)

In [ ]:
number_of_measurements = np.sum(reference)
number_of_measurements

In [ ]:
electronfactors.plot_model(width_data[reference], length_data[reference], factor_data[reference])

In [ ]:
weighted_difference_squares = (
    (width_data[reference] - width) ** 2 + 
    0.5 * (length_data[reference] - length) ** 2
)
closest_cutout = np.argmin(weighted_difference_squares)
closest_width = width_data[reference][closest_cutout]
closest_length = length_data[reference][closest_cutout]
closest_factor = factor_data[reference][closest_cutout]
print('')
print(
    "Closest cutout within data\n"
    "  Width: {0:0.2f} cm\n"
    "  Length: {1:0.2f} cm\n"
    "  Factor: {2:0.3f}".format(closest_width, closest_length, closest_factor)
)